# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


#### Optional: Run this cell to see available notebook commands ("magics").


In [ ]:
%help

####  Run this cell to set up and start your interactive session.


In [1]:
%idle_timeout 2880
%glue_version 4.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.5 
Current idle_timeout is None minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 4.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 5
Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 5
Idle Timeout: 2880
Session ID: 26f23661-f728-4b54-8cfd-f93d4322f3e1
Applying the following default arguments:
--glue_kernel_version 1.0.5
--enable-glue-datacatalog true
Waiting for session 26f23661-f728-4b54-8cfd-f93d4322f3e1 to get into ready status...
Session 26f23661-f728-4b54-8cfd-f93d4322f3e1 ha

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, explode
from pyspark.sql.types import IntegerType, StructType, StructField, StringType, ArrayType

In [3]:
spark = SparkSession.builder.appName("dataeng-modulo-2").getOrCreate()

In [4]:
data = [
    ("João", [{"curso": "Matemática", "nota": 85}, {"curso": "História", "nota": 90}]),
    ("Maria", [{"curso": "Matemática", "nota": 95}, {"curso": "História", "nota": 80}])
]

In [5]:
schema = StructType([
    StructField("nome", StringType(), True),
    StructField("cursos", ArrayType(StructType([
        StructField("curso", StringType(), True),
        StructField("nota", IntegerType(), True)
    ])), True)
])

In [6]:
df = spark.createDataFrame(data, schema)


In [7]:
df.show(truncate=False)

+-----+----------------------------------+
|nome |cursos                            |
+-----+----------------------------------+
|João |[{Matemática, 85}, {História, 90}]|
|Maria|[{Matemática, 95}, {História, 80}]|
+-----+----------------------------------+


# 2. Explodindo o Array para Linhas Individuais

In [8]:
df_exploded = df.withColumn("curso", explode(df["cursos"]))
df_exploded = df_exploded.select("nome", col("curso.curso"), col("curso.nota"))
df_exploded.show()

+-----+----------+----+
| nome|     curso|nota|
+-----+----------+----+
| João|Matemática|  85|
| João|  História|  90|
|Maria|Matemática|  95|
|Maria|  História|  80|
+-----+----------+----+


# 3. Definindo uma UDF para Calcular um Bônus na Nota


In [9]:
@udf(IntegerType())
def calcular_bonus(nota):
    return nota + 5

In [10]:
df_bonus = df_exploded.withColumn("nota_bonus", calcular_bonus(df_exploded["nota"]))
df_bonus.show()


+-----+----------+----+----------+
| nome|     curso|nota|nota_bonus|
+-----+----------+----+----------+
| João|Matemática|  85|        90|
| João|  História|  90|        95|
|Maria|Matemática|  95|       100|
|Maria|  História|  80|        85|
+-----+----------+----+----------+


# 4. Aplicação de Pivot

In [11]:
df_pivot_bonus = df_bonus.groupBy("nome").pivot("curso").agg({"nota_bonus": "max"})
df_pivot_bonus.show()

+-----+--------+----------+
| nome|História|Matemática|
+-----+--------+----------+
|Maria|      85|       100|
| João|      95|        90|
+-----+--------+----------+


# 5. Rollup para Agregações Hierárquicas


In [12]:
df_rollup = df_exploded.rollup("nome", "curso").agg({"nota": "avg"}).orderBy("nome", "curso")
df_rollup.show()

+-----+----------+---------+
| nome|     curso|avg(nota)|
+-----+----------+---------+
| null|      null|     87.5|
| João|      null|     87.5|
| João|  História|     90.0|
| João|Matemática|     85.0|
|Maria|      null|     87.5|
|Maria|  História|     80.0|
|Maria|Matemática|     95.0|
+-----+----------+---------+


# 6. Cube para Agregações Multidimensionais


In [13]:
df_cube = df_exploded.cube("nome", "curso").agg({"nota": "avg"}).orderBy("nome", "curso")
df_cube.show()

+-----+----------+---------+
| nome|     curso|avg(nota)|
+-----+----------+---------+
| null|      null|     87.5|
| null|  História|     85.0|
| null|Matemática|     90.0|
| João|      null|     87.5|
| João|  História|     90.0|
| João|Matemática|     85.0|
|Maria|      null|     87.5|
|Maria|  História|     80.0|
|Maria|Matemática|     95.0|
+-----+----------+---------+


# Encerrando a SparkSession


In [ ]:
spark.stop()